In [ ]:
import tkinter as tk
from tkinter import scrolledtext
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load your dataset containing titles and corresponding comments
df = pd.read_csv('chatbot_dataset.csv')

# Handle null values
df.dropna(inplace=True)

# Preprocess the text data
def preprocess_text(text):
    # Add your preprocessing steps here
    return text.lower()  # Example: Convert text to lowercase

# Apply preprocessing to title and comment columns
df['Title'] = df['Title'].apply(preprocess_text)
df['Top Comment 1'] = df['Top Comment 1'].apply(preprocess_text)

# Concatenate title and comment columns to create corpus
df['corpus'] = df['Title'] + ' ' + df['Top Comment 1']

# Initialize TfidfVectorizer to convert text into TF-IDF vectors
vectorizer = TfidfVectorizer()

# Fit and transform the corpus
X = vectorizer.fit_transform(df['corpus'])

# Function to generate response
def generate_response(user_input, df, X, vectorizer):
    # Preprocess user input
    user_input = preprocess_text(user_input)
    
    # Transform user input into TF-IDF vector
    user_input_vector = vectorizer.transform([user_input])
    
    # Calculate cosine similarity between user input and each document
    similarities = cosine_similarity(user_input_vector, X)
    
    # Get index of the most similar document
    most_similar_idx = similarities.argmax()
    
    # Retrieve the corresponding response from the dataset
    response = df.iloc[most_similar_idx]['Top Comment 1']
    
    return response

# Function to handle user input and display bot response
def chat():
    user_input = user_entry.get()
    response = generate_response(user_input, df, X, vectorizer)
    chat_history.insert(tk.END, "You: " + user_input + "\n")
    chat_history.insert(tk.END, "Bot: " + response + "\n\n")
    user_entry.delete(0, tk.END)  # Clear user input field

# Create main window
window = tk.Tk()
window.title("GuidChat - Career Guidance Chatbot")

# Create chat history display area
chat_history = scrolledtext.ScrolledText(window, wrap=tk.WORD, width=50, height=20)
chat_history.grid(row=0, column=0, columnspan=2, padx=10, pady=10)

# Create user input field
user_entry = tk.Entry(window, width=40)
user_entry.grid(row=1, column=0, padx=10, pady=5)

# Create send button
send_button = tk.Button(window, text="Send", command=chat)
send_button.grid(row=1, column=1, padx=10, pady=5)

# Function to handle closing the window
def close_window():
    window.destroy()

# Bind the close_window function to the window close event
window.protocol("WM_DELETE_WINDOW", close_window)

# Start the GUI event loop
window.mainloop()
